# How to manage memory

:::info Prerequisites

This guide assumes familiarity with the following:

- [Chatbots](/docs/tutorials/chatbot)

:::

A key feature of chatbots is their ability to use content of previous conversation turns as context. This state management can take several forms, including:

- Simply stuffing previous messages into a chat model prompt.
- The above, but trimming old messages to reduce the amount of distracting information the model has to deal with.
- More complex modifications like synthesizing summaries for long running conversations.

We’ll go into more detail on a few techniques below!

## Setup

You’ll need to install a few packages, and set any LLM API keys:

Let’s also set up a chat model that we’ll use for the below examples:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs />
```

## Message passing

The simplest form of memory is simply passing chat history messages into a chain. Here’s an example:

In [2]:
import { HumanMessage, AIMessage } from "@langchain/core/messages";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "You are a helpful assistant. Answer all questions to the best of your ability.",
  ],
  new MessagesPlaceholder("messages"),
]);

const chain = prompt.pipe(llm);

await chain.invoke({
  messages: [
    new HumanMessage(
      "Translate this sentence from English to French: I love programming."
    ),
    new AIMessage("J'adore la programmation."),
    new HumanMessage("What did you just say?"),
  ],
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: `I said "J'adore la programmation," which means "I love programming" in French.`,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: `I said "J'adore la programmation," which means "I love programming" in French.`,
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 21, promptTokens: 61, totalTokens: 82 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

We can see that by passing the previous conversation into a chain, it can use it as context to answer questions. This is the basic concept underpinning chatbot memory - the rest of the guide will demonstrate convenient techniques for passing or reformatting messages.

## Chat history

It’s perfectly fine to store and pass messages directly as an array, but we can use LangChain’s built-in message history class to store and load messages as well. Instances of this class are responsible for storing and loading chat messages from persistent storage. LangChain integrates with many providers but for this demo we will use an ephemeral demo class.

Here’s an example of the API:

In [3]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";

const demoEphemeralChatMessageHistory = new ChatMessageHistory();

await demoEphemeralChatMessageHistory.addMessage(
  new HumanMessage(
    "Translate this sentence from English to French: I love programming."
  )
);

await demoEphemeralChatMessageHistory.addMessage(
  new AIMessage("J'adore la programmation.")
);

await demoEphemeralChatMessageHistory.getMessages();

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Translate this sentence from English to French: I love programming.",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Translate this sentence from English to French: I love programming.",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "J'adore la programmation.",
      tool_calls: [],
      invalid_tool_calls: [],
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "J'adore la programmation.",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {},
    tool_calls: [],
    invalid_tool_calls: []
  }
]

We can use it directly to store conversation turns for our chain:

In [4]:
await demoEphemeralChatMessageHistory.clear();

const input1 =
  "Translate this sentence from English to French: I love programming.";

await demoEphemeralChatMessageHistory.addMessage(new HumanMessage(input1));

const response = await chain.invoke({
  messages: await demoEphemeralChatMessageHistory.getMessages(),
});

await demoEphemeralChatMessageHistory.addMessage(response);

const input2 = "What did I just ask you?";

await demoEphemeralChatMessageHistory.addMessage(new HumanMessage(input2));

await chain.invoke({
  messages: await demoEphemeralChatMessageHistory.getMessages(),
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: 'You just asked me to translate the sentence "I love programming" from English to French.',
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: 'You just asked me to translate the sentence "I love programming" from English to French.',
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 18, promptTokens: 73, totalTokens: 91 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

## Automatic history management

The previous examples pass messages to the chain explicitly. This is a completely acceptable approach, but it does require external management of new messages. LangChain also includes an wrapper for LCEL chains that can handle this process automatically called `RunnableWithMessageHistory`.

To show how it works, let’s slightly modify the above prompt to take a final `input` variable that populates a `HumanMessage` template after the chat history. This means that we will expect a `chat_history` parameter that contains all messages BEFORE the current messages instead of all messages:

In [6]:
const runnableWithMessageHistoryPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "You are a helpful assistant. Answer all questions to the best of your ability.",
  ],
  new MessagesPlaceholder("chat_history"),
  ["human", "{input}"],
]);

const chain2 = runnableWithMessageHistoryPrompt.pipe(llm);

We’ll pass the latest input to the conversation here and let the `RunnableWithMessageHistory` class wrap our chain and do the work of appending that `input` variable to the chat history.

Next, let’s declare our wrapped chain:

In [7]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

const demoEphemeralChatMessageHistoryForChain = new ChatMessageHistory();

const chainWithMessageHistory = new RunnableWithMessageHistory({
  runnable: chain2,
  getMessageHistory: (_sessionId) => demoEphemeralChatMessageHistoryForChain,
  inputMessagesKey: "input",
  historyMessagesKey: "chat_history",
});

This class takes a few parameters in addition to the chain that we want to wrap:

- A factory function that returns a message history for a given session id. This allows your chain to handle multiple users at once by loading different messages for different conversations.
- An `inputMessagesKey` that specifies which part of the input should be tracked and stored in the chat history. In this example, we want to track the string passed in as input.
- A `historyMessagesKey` that specifies what the previous messages should be injected into the prompt as. Our prompt has a `MessagesPlaceholder` named `chat_history`, so we specify this property to match.
  (For chains with multiple outputs) an `outputMessagesKey` which specifies which output to store as history. This is the inverse of `inputMessagesKey`.

We can invoke this new chain as normal, with an additional `configurable` field that specifies the particular `sessionId` to pass to the factory function. This is unused for the demo, but in real-world chains, you’ll want to return a chat history corresponding to the passed session:

In [8]:
await chainWithMessageHistory.invoke(
  {
    input:
      "Translate this sentence from English to French: I love programming.",
  },
  { configurable: { sessionId: "unused" } }
);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: `The translation of "I love programming" in French is "J'adore la programmation."`,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: `The translation of "I love programming" in French is "J'adore la programmation."`,
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 20, promptTokens: 39, totalTokens: 59 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [9]:
await chainWithMessageHistory.invoke(
  {
    input: "What did I just ask you?",
  },
  { configurable: { sessionId: "unused" } }
);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: 'You just asked for the translation of the sentence "I love programming" from English to French.',
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: 'You just asked for the translation of the sentence "I love programming" from English to French.',
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 19, promptTokens: 74, totalTokens: 93 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

## Modifying chat history

Modifying stored chat messages can help your chatbot handle a variety of situations. Here are some examples:

### Trimming messages

LLMs and chat models have limited context windows, and even if you’re not directly hitting limits, you may want to limit the amount of distraction the model has to deal with. One solution is to only load and store the most recent `n` messages. Let’s use an example history with some preloaded messages:

In [10]:
await demoEphemeralChatMessageHistory.clear();

await demoEphemeralChatMessageHistory.addMessage(
  new HumanMessage("Hey there! I'm Nemo.")
);

await demoEphemeralChatMessageHistory.addMessage(new AIMessage("Hello!"));

await demoEphemeralChatMessageHistory.addMessage(
  new HumanMessage("How are you today?")
);

await demoEphemeralChatMessageHistory.addMessage(new AIMessage("Fine thanks!"));

await demoEphemeralChatMessageHistory.getMessages();

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hey there! I'm Nemo.",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hey there! I'm Nemo.",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hello!",
      tool_calls: [],
      invalid_tool_calls: [],
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello!",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {},
    tool_calls: [],
    invalid_tool_calls: []
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "How are you today?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "How are you today?",
    name: undefi

Let’s use this message history with the `RunnableWithMessageHistory` chain we declared above:

In [11]:
const chainWithMessageHistory2 = new RunnableWithMessageHistory({
  runnable: chain2,
  getMessageHistory: (_sessionId) => demoEphemeralChatMessageHistory,
  inputMessagesKey: "input",
  historyMessagesKey: "chat_history",
});

await chainWithMessageHistory2.invoke(
  {
    input: "What's my name?",
  },
  { configurable: { sessionId: "unused" } }
);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Your name is Nemo!",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Your name is Nemo!",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 6, promptTokens: 66, totalTokens: 72 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

We can see the chain remembers the preloaded name.

But let’s say we have a very small context window, and we want to trim the number of messages passed to the chain to only the 2 most recent ones. We can use the `clear` method to remove messages and re-add them to the history. We don’t have to, but let’s put this method at the front of our chain to ensure it’s always called:

In [12]:
import {
  RunnablePassthrough,
  RunnableSequence,
} from "@langchain/core/runnables";

const trimMessages = async (_chainInput: Record<string, any>) => {
  const storedMessages = await demoEphemeralChatMessageHistory.getMessages();
  if (storedMessages.length <= 2) {
    return false;
  }
  await demoEphemeralChatMessageHistory.clear();
  for (const message of storedMessages.slice(-2)) {
    demoEphemeralChatMessageHistory.addMessage(message);
  }
  return true;
};

const chainWithTrimming = RunnableSequence.from([
  RunnablePassthrough.assign({ messages_trimmed: trimMessages }),
  chainWithMessageHistory2,
]);

Let’s call this new chain and check the messages afterwards:

In [13]:
await chainWithTrimming.invoke(
  {
    input: "Where does P. Sherman live?",
  },
  { configurable: { sessionId: "unused" } }
);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: 'P. Sherman is a fictional character who lives at 42 Wallaby Way, Sydney, from the movie "Finding Nem'... 3 more characters,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: 'P. Sherman is a fictional character who lives at 42 Wallaby Way, Sydney, from the movie "Finding Nem'... 3 more characters,
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 26, promptTokens: 53, totalTokens: 79 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [14]:
await demoEphemeralChatMessageHistory.getMessages();

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What's my name?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What's my name?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Your name is Nemo!",
      tool_calls: [],
      invalid_tool_calls: [],
      additional_kwargs: { function_call: undefined, tool_calls: undefined },
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Your name is Nemo!",
    name: undefined,
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {
      tokenUsage: { completionTokens: 6, promptTokens: 66, totalTokens: 72 },
      finish_reason: "stop"
    },
    tool_calls: [],
    invalid_tool_calls: []
  },
  HumanMessage {
    lc_serializable: true

And we can see that our history has removed the two oldest messages while still adding the most recent conversation at the end. The next time the chain is called, `trimMessages` will be called again, and only the two most recent messages will be passed to the model. In this case, this means that the model will forget the name we gave it the next time we invoke it:

In [15]:
await chainWithTrimming.invoke(
  {
    input: "What is my name?",
  },
  { configurable: { sessionId: "unused" } }
);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "I'm sorry, I don't have access to your personal information. Can I help you with anything else?",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "I'm sorry, I don't have access to your personal information. Can I help you with anything else?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 22, promptTokens: 73, totalTokens: 95 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [16]:
await demoEphemeralChatMessageHistory.getMessages();

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Where does P. Sherman live?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Where does P. Sherman live?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: 'P. Sherman is a fictional character who lives at 42 Wallaby Way, Sydney, from the movie "Finding Nem'... 3 more characters,
      tool_calls: [],
      invalid_tool_calls: [],
      additional_kwargs: { function_call: undefined, tool_calls: undefined },
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: 'P. Sherman is a fictional character who lives at 42 Wallaby Way, Sydney, from the movie "Finding Nem'... 3 more characters,
    name: undefined,
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
  

### Summary memory

We can use this same pattern in other ways too. For example, we could use an additional LLM call to generate a summary of the conversation before calling our chain. Let’s recreate our chat history and chatbot chain:

In [17]:
await demoEphemeralChatMessageHistory.clear();

await demoEphemeralChatMessageHistory.addMessage(
  new HumanMessage("Hey there! I'm Nemo.")
);

await demoEphemeralChatMessageHistory.addMessage(new AIMessage("Hello!"));

await demoEphemeralChatMessageHistory.addMessage(
  new HumanMessage("How are you today?")
);

await demoEphemeralChatMessageHistory.addMessage(new AIMessage("Fine thanks!"));

In [19]:
const runnableWithSummaryMemoryPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with.",
  ],
  new MessagesPlaceholder("chat_history"),
  ["human", "{input}"],
]);

const summaryMemoryChain = runnableWithSummaryMemoryPrompt.pipe(llm);

const chainWithMessageHistory3 = new RunnableWithMessageHistory({
  runnable: summaryMemoryChain,
  getMessageHistory: (_sessionId) => demoEphemeralChatMessageHistory,
  inputMessagesKey: "input",
  historyMessagesKey: "chat_history",
});

And now, let’s create a function that will distill previous interactions into a summary. We can add this one to the front of the chain too:

In [22]:
const summarizeMessages = async (_chainInput: Record<string, any>) => {
  const storedMessages = await demoEphemeralChatMessageHistory.getMessages();
  if (storedMessages.length === 0) {
    return false;
  }
  const summarizationPrompt = ChatPromptTemplate.fromMessages([
    new MessagesPlaceholder("chat_history"),
    [
      "user",
      "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
    ],
  ]);
  const summarizationChain = summarizationPrompt.pipe(llm);
  const summaryMessage = await summarizationChain.invoke({
    chat_history: storedMessages,
  });
  await demoEphemeralChatMessageHistory.clear();
  demoEphemeralChatMessageHistory.addMessage(summaryMessage);
  return true;
};

const chainWithSummarization = RunnableSequence.from([
  RunnablePassthrough.assign({
    messages_summarized: summarizeMessages,
  }),
  chainWithMessageHistory3,
]);

Let’s see if it remembers the name we gave it:

In [23]:
await chainWithSummarization.invoke(
  {
    input: "What did I say my name was?",
  },
  {
    configurable: { sessionId: "unused" },
  }
);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: 'You introduced yourself as "Nemo."',
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: 'You introduced yourself as "Nemo."',
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 8, promptTokens: 87, totalTokens: 95 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [24]:
await demoEphemeralChatMessageHistory.getMessages();

[
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "The conversation consists of a greeting from someone named Nemo and a general inquiry about their we"... 86 more characters,
      tool_calls: [],
      invalid_tool_calls: [],
      additional_kwargs: { function_call: undefined, tool_calls: undefined },
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "The conversation consists of a greeting from someone named Nemo and a general inquiry about their we"... 86 more characters,
    name: undefined,
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {
      tokenUsage: { completionTokens: 34, promptTokens: 62, totalTokens: 96 },
      finish_reason: "stop"
    },
    tool_calls: [],
    invalid_tool_calls: []
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What did I say my name was?",
      additional_kwargs: {},
      response_meta

Note that invoking the chain again will generate another summary generated from the initial summary plus new messages and so on. You could also design a hybrid approach where a certain number of messages are retained in chat history while others are summarized.

## Next steps

You've now learned how to manage memory in your chatbots

Next, check out some of the other guides in this section, such as [how to add retrieval to your chatbot](/docs/how_to/chatbots_retrieval).